In [1]:
import pandas as pd
import altair as alt

In [26]:
churn = pd.read_csv('BankChurners.csv')

In [7]:
features = ['Attrition_Flag',
            'Customer_Age',
            'Gender',
            'Marital_Status',
            'Income_Category',
            'Total_Trans_Amt',
            'Credit_Limit',
            'Months_Inactive_12_mon']

churn = churn[features].sample(n=5000, ignore_index=True)

churn.head()

,Attrition_Flag,Customer_Age,Gender,Marital_Status,Income_Category,Total_Trans_Amt,Credit_Limit,Months_Inactive_12_mon
0,Existing Customer,47,M,Unknown,$40K - $60K,3480,10960.0,3
1,Existing Customer,27,F,Married,Less than $40K,5176,1893.0,3
2,Existing Customer,43,M,Unknown,$40K - $60K,5021,4878.0,3
3,Existing Customer,63,M,Single,$80K - $120K,4456,9033.0,2
4,Attrited Customer,44,F,Single,$40K - $60K,2558,4196.0,3


## Data Aggregation

altair permite a agregação de dados enquanto cria a visualização. Isso nos salva algumas operações que são tipicamente feitas com a analise de dados e manipulação de bibliotecas como o pandas. 

Por exemplo, nos podemos criar um bar plot que demostra a media do limite de credito para cada income_category 

In [11]:
alt.Chart(churn).mark_bar().encode(x='Income_Category', y='mean(Credit_Limit):Q').properties(height=300, width=400)

alt.Chart(...)

O que foi passado no Y é a mesma que usar um groupby

In [12]:
churn.groupby('Income_Category').agg(Mean_Credit_Limit = ('Credit_Limit','mean'))

,Mean_Credit_Limit
Income_Category,
$120K +,19646.414286
$40K - $60K,5234.067191
$60K - $80K,11157.865680
$80K - $120K,15666.623202
Less than $40K,3750.566742
Unknown,9382.312455


In [14]:
# eu quem fiz
churn.groupby('Income_Category')['Credit_Limit'].mean()

Income_Category
$120K +           19646.414286
$40K - $60K        5234.067191
$60K - $80K       11157.865680
$80K - $120K      15666.623202
Less than $40K     3750.566742
Unknown            9382.312455
Name: Credit_Limit, dtype: float64

## Filtragem

O Altair também fornece funções para filtrar dados para que possamos criar gráficos mais focados ou específicos. Existem vários predicados que são passados para a transform_filter função. Cada predicado aplica um método diferente de filtragem.

Por exemplo, o de predicados permite filtrar linhas com base em uma lista de valores. O código a seguir cria um gráfico de barras que mostra o limite de crédito médio para cada estado civil fornecido na lista especificada pelo oneOf parâmetro.

In [22]:
alt.Chart(churn).mark_bar(width=50).encode(x='Marital_Status', y='mean(Credit_Limit):Q').transform_filter(alt.FieldOneOfPredicate(field='Marital_Status', oneOf=['Single','Married','Divorced'])).properties(height=300, width=500)

alt.Chart(...)

## Filtragem dinâmica

Também podemos implementar filtragem dinâmica com Altair. Permite vincular vários gráficos com um filtro compartilhado.

Primeiro criamos um objeto de seleção que captura as interações do usuário. Como o próprio nome sugere, é usado para selecionar valores na visualização. O objeto de seleção pode ser conectado a uma legenda ou outro gráfico.

Em outras palavras, especificamos uma condição em um gráfico ou em uma legenda usando o objeto de seleção. Em seguida, os valores são filtrados com base nessa seleção.

Ficará mais claro com um exemplo.

In [24]:
selection = alt.selection_multi(
    fields=['Attrition_Flag'], bind='legend'
)

alt.Chart(churn).mark_circle(size=50).encode(
    x='Credit_Limit',
    y='Total_Trans_Amt',
    color='Attrition_Flag',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).properties(
    height=350, width=500
).add_selection(
    selection
)

alt.Chart(...)